In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [5]:
cities_data = pd.read_csv("resources\city_weather.csv")
cities_data = cities_data.drop('Unnamed: 0', axis=1)
cities_data.head()

,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed
0,yumen,40.2833,97.2000,89.82,10,19,13.40
1,bredasdorp,-34.5322,20.0403,69.12,37,0,5.75
2,coihaique,-45.5752,-72.0662,35.55,80,0,4.61
3,constitucion,-35.3333,-72.4167,41.65,72,1,9.04
4,nanortalik,60.1432,-45.2371,45.77,80,100,2.75


In [9]:
# Store lat and lon locations
locations = cities_data[['Lat', 'Lon']]

# Store humidity values
humidity = cities_data['Humidity']

# Configure maps
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

# Plot heatmaps
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=humidity.max())

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Narrow the selection of data

data_selection = (
                (cities_data['Temp'] > 70) &
                (cities_data['Temp'] < 80) &
                (cities_data['Wind Speed'] < 10) &
                (cities_data['Cloudiness'] == 0)
)

ideal_cities = cities_data.loc[data_selection]
ideal_cities

,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed
51,arraial do cabo,-22.9661,-42.0278,71.46,73,0,6.91
176,macae,-22.3708,-41.7869,71.31,73,0,6.91
184,maceio,-9.6658,-35.7353,74.64,100,0,3.44
359,rundu,-17.9333,19.7667,72.12,19,0,8.10
365,cayenne,4.9333,-52.3333,77.04,94,0,2.30
384,henties bay,-22.1160,14.2845,74.16,38,0,7.85


In [20]:
# Add column to dataframe
ideal_cities['Hotel name'] = ""

# Use Google API to get hotel data
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters for search
params = {
    "radius" : 5000,
    "keyword" : "hotel",
    "key" : g_key
}

for index, row in ideal_cities.iterrows():
    params["location"] = f"{row['Lat']},{row['Lon']}"
    
    response = requests.get(base_url, params=params).json()
    # Returns first hotel in Google's list. May not necessarily be the closest!
    try:
        hotel = response["results"][0]
        ideal_cities.loc[index, 'Hotel name'] = hotel["name"]
    except KeyError:
        print(f"Hmm, no hotel found in {row['City']}")
        print(response)

ideal_cities

<ipython-input-20-19ae5d176d03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities['Hotel name'] = ""
C:\Users\mail\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Hotel name
51,arraial do cabo,-22.9661,-42.0278,71.46,73,0,6.91,Orlanova Boutique Hotel
176,macae,-22.3708,-41.7869,71.31,73,0,6.91,Royal Macaé Palace Hotel
184,maceio,-9.6658,-35.7353,74.64,100,0,3.44,Jatiúca Hotel & Resort
359,rundu,-17.9333,19.7667,72.12,19,0,8.10,Hakusembe River Lodge
365,cayenne,4.9333,-52.3333,77.04,94,0,2.30,Grand Hotel Montabo
384,henties bay,-22.1160,14.2845,74.16,38,0,7.85,De Duine Hotel


In [26]:
# sample code used from starter notebook
# country removed as not stored in dataframe

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]
hover_text = [row['Hotel name'] for index,row in ideal_cities.iterrows()]
locations = ideal_cities[["Lat", "Lon"]]

hotel_marks = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hover_text)
fig.add_layer(hotel_marks)
fig

Figure(layout=FigureLayout(height='420px'))